In [28]:
import torch
from torch.optim import SGD
import plotly.express as px
import pandas as pd

# Задание 1

In [5]:
import numpy as np
def sample_data():
    count = 10000
    rand = np.random.RandomState(0)
    a = 0.3 + 0.1 * rand.randn(count)
    b = 0.8 + 0.05 * rand.randn(count)
    mask = rand.rand(count) < 0.5
    samples = np.clip(a * mask + b * (1 - mask), 0.0, 1.0)
    return np.digitize(samples, np.linspace(0.0, 1.0, 100))

In [18]:
def p(theta, x):
    return torch.exp(theta[x]) / torch.sum(torch.exp(theta))

def J(output):
    return -torch.sum(torch.log(output))

In [48]:
theta = torch.zeros(100, requires_grad=True)
sgd = SGD([theta], lr=0.03)

num_epochs = 20
losses = []
sample = sample_data()
size_train = int(len(sample) * 0.8)
size_test = len(sample) - size_train
train, test = torch.utils.data.random_split(sample, (size_train, size_test))

x_train = torch.tensor(train, dtype=torch.long)
x_test = torch.tensor(test, dtype=torch.long)
dataset = torch.utils.data.DataLoader(x_train, batch_size=1000, shuffle=True)
train_losses = []
test_losses = []
for i in range(num_epochs):
    for x1 in dataset:
        sgd.zero_grad()
        output = p(theta, x1)
        loss = J(output)
        losses.append(float(loss))
        loss.backward()
        sgd.step()
    train_losses.append(float(J(p(theta, x_train)).double()) / size_train)
    test_losses.append(float(J(p(theta, x_test)).double()) / size_test)

In [53]:
df = pd.DataFrame()
df['loss'] = train_losses + test_losses
df['data'] = ['train'] * num_epochs + ['test'] * num_epochs
df['iter'] = list(range(num_epochs)) + list(range(num_epochs))

In [55]:
px.line(df, x='iter', y='loss', color='data')

In [79]:
predicted = p(theta, range(100)).tolist()
expected = np.zeros(100)
for x in sample[:1000]:
    expected[x] += 1
expected /= 1000
expected = list(expected)

df = pd.DataFrame()
df['prob'] = expected + predicted
df['data'] = ['expected'] * 100 + ['predicted'] * 100
df['val'] = list(range(100)) + list(range(100))

In [80]:
px.line(df, x='val', y='prob', color='data')

# Задание 2